# 🗂 Weak supervision in multi-label text classification tasks

WORK IN PROGRESS: This tutorial is a work in progress and you can expect some changes within the next few releases.
We will showcase new features here as soon as they are available.

In this tutorial we use Rubrix and weak supervision to tackle two multi-label classification datasets:

- The first dataset is a curated version of [**GoEmotions**](https://huggingface.co/datasets/go_emotions), a dataset intended for **multi-label emotion classification**.
- We inspect the dataset in Rubrix, come up with good heuristics, and combine them with a label model to train a **weakly supervised Hugging Face transformer**.
- In the second dataset, we [**categorize research papers**](https://www.kaggle.com/shivanandmn/multilabel-classification-dataset) by topic based on their titles, which is a **multi-label topic classification** problem.
- We repeat the process of finding good heuristics, combine them with a label model and train a **lightweight downstream model using sklearn** in the end.

<video width="100%" controls><source src="../_static/tutorials/weak-supervision-multi-label/ws_multi-label.mp4" type="video/mp4"></video>

<div class="alert alert-info">

Note

If you are new to weak supervision, check out our [weak supervision guide](../guides/weak-supervision.ipynb) and our first [weak supervision tutorial](weak-supervision-with-rubrix.ipynb).
    
</div>

## Setup

Rubrix, is a free and open-source tool to explore, annotate, and monitor data for NLP projects.

If you are new to Rubrix, check out the [Github repository](https://github.com/recognai/rubrix) ⭐.

If you have not installed and launched Rubrix yet, check the [Setup and Installation guide](../getting_started/setup&installation.rst).

For this tutorial we also need some third party libraries that can be installed via pip:

In [ ]:
%pip install datasets "transformers[torch]" scikit-multilearn ipywidgets -qqq

## GoEmotions

The original [GoEmotions](https://huggingface.co/datasets/go_emotions) is a challenging dataset intended for multi-label emotion classification.
For this tutorial, we simplify it a bit by selecting only 6 out of the 28 emotions: *admiration, annoyance, approval, curiosity, gratitude, optimism*.
We also try to accentuate the multi-label part of the dataset by down-sampling the examples that are classified with only one label.
See Appendix A for all the details of this preprocessing step.

### Define rules

Let us start by downloading our curated version of the dataset from the Hugging Face Hub, and log it to Rubrix:

In [74]:
import rubrix as rb
from datasets import load_dataset

# Download preprocessed dataset
ds_rb = rb.read_datasets(
    load_dataset("rubrix/go_emotions_multi-label", split="train"),
    task="TextClassification"
)

2022-05-03 11:40:18.863 | WARNING  | datasets.builder:_create_builder_config:378 - Using custom data configuration rubrix--go_emotions_multi-label-be9f0b20d2f10f30
2022-05-03 11:40:18.865 | WARNING  | datasets.builder:download_and_prepare:531 - Reusing dataset parquet (/home/david/.cache/huggingface/datasets/parquet/rubrix--go_emotions_multi-label-be9f0b20d2f10f30/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


In [15]:
# Log dataset to Rubrix to find good heuristics
rb.log(ds_rb, name="go_emotions")

  0%|          | 0/4208 [00:00<?, ?it/s]

4208 records logged to http://localhost:6900/datasets/rubrix/go_emotions


BulkResponse(dataset='go_emotions', processed=4208, failed=0)

After uploading the dataset, we can explore and inspect it to find good heuristic rules.
For this we highly recommend the dedicated [*Define rules* mode](../reference/webapp/define_rules.md) of the Rubrix web app, that allows you to quickly iterate over heuristic rules, compute their metrics and save them.

Here we copy our rules found via the web app to the notebook for you to easily follow along the tutorial.

In [75]:
from rubrix.labeling.text_classification import Rule

# Define our heuristic rules, they can surely be improved
rules = [
    Rule("thank*", "gratitude"),
    Rule("appreciate", "gratitude"),
    Rule("text:(thanks AND good)", ["admiration", "gratitude"]),
    Rule("advice", "admiration"),
    Rule("amazing", "admiration"),
    Rule("awesome", "admiration"),
    Rule("impressed", "admiration"),
    Rule("text:(good AND (point OR call OR idea OR job))", "admiration"),
    Rule("legend", "admiration"),
    Rule("exactly", "approval"),
    Rule("agree", "approval"),
    Rule("yeah", "approval"),
    Rule("suck", "annoyance"),
    Rule("pissed", "annoyance"),
    Rule("annoying", "annoyance"),
    Rule("ruined", "annoyance"),
    Rule("hoping", "optimism"),
    Rule('text:("good luck")', "optimism"),
    Rule('"nice day"', "optimism"),
    Rule('"what is"', "curiosity"),
    Rule('"can you"', "curiosity"),
    Rule('"would you"', "curiosity"),
]

We go on and apply these heuristic rules to our dataset creating our weak label matrix.
Since we are dealing with a multi-label classification task, the weak label matrix will have 3 dimensions.

> Dimensions of the weak multi label matrix: *number of records* x *number of rules* x *number of labels* 

It will be filled with 0 and 1, depending on if the rule voted for the respective label or not.
If the rule abstained for a given record, the matrix will be filled with -1. 

In [76]:
from rubrix.labeling.text_classification import WeakMultiLabels

# Compute the weak labels for our dataset given the rules.
# If your dataset already contains rules you can omit the rules argument.
weak_labels = WeakMultiLabels("go_emotions", rules=rules)

Preparing rules:   0%|          | 0/22 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/4208 [00:00<?, ?it/s]

Filling weak label matrix:   0%|          | 0/4208 [00:00<?, ?it/s]

We can call the `weak_labels.summary()` method to check the precision of each rule as well as our total coverage of the dataset.

In [77]:
# Check coverage/precision of our rules
weak_labels.summary()

,label,coverage,annotated_coverage,overlaps,correct,incorrect,precision
thank*,{gratitude},0.196768,0.196237,0.037785,73,0,1.000000
appreciate,{gratitude},0.016160,0.021505,0.009506,7,1,0.875000
text:(thanks AND good),"{admiration, gratitude}",0.007842,0.010753,0.007605,8,0,1.000000
advice,{admiration},0.008317,0.008065,0.006654,3,0,1.000000
amazing,{admiration},0.025428,0.021505,0.003565,8,0,1.000000
awesome,{admiration},0.025190,0.034946,0.006179,12,1,0.923077
impressed,{admiration},0.002139,0.005376,0.000000,2,0,1.000000
text:(good AND (point OR call OR idea OR job)),{admiration},0.008555,0.018817,0.002376,7,0,1.000000
legend,{admiration},0.001901,0.002688,0.000475,1,0,1.000000
exactly,{approval},0.004278,0.002688,0.001188,1,0,1.000000


### Create training set

When we are happy with our heuristics, it is time to combine them and compute weak labels for the training of our downstream model.
Here we will use the simple `MajorityVoter`, that in the multi-label case, sets the probability of a label to 0 or 1 depending on whether at least one non-abstaining rule voted for the respective label or not.

In [78]:
from rubrix.labeling.text_classification import MajorityVoter

# Use the majority voter as the label model
label_model = MajorityVoter(weak_labels)

From our label model we get the training records together with its weak labels and probabilities.
We will use the weak labels with a probability greater than 0.5 as labels for our training, and hence copy them to the `annotation` property of our records.

In [79]:
print(label_model.score(output_str=True))

              precision    recall  f1-score   support

  admiration       0.97      0.61      0.75        54
   annoyance       0.93      0.72      0.81        18
    approval       0.71      0.52      0.60        23
   curiosity       1.00      0.39      0.56        18
   gratitude       0.99      0.96      0.97        76
    optimism       0.73      0.47      0.57        17

   micro avg       0.93      0.71      0.80       206
   macro avg       0.89      0.61      0.71       206
weighted avg       0.93      0.71      0.79       206
 samples avg       0.94      0.78      0.83       206



#### compare to Snorkel model

In [80]:
label_model.score()

{'admiration': {'precision': 0.9705882352941176,
  'recall': 0.6111111111111112,
  'f1-score': 0.7499999999999999,
  'support': 54},
 'annoyance': {'precision': 0.9285714285714286,
  'recall': 0.7222222222222222,
  'f1-score': 0.8125000000000001,
  'support': 18},
 'approval': {'precision': 0.7058823529411765,
  'recall': 0.5217391304347826,
  'f1-score': 0.6,
  'support': 23},
 'curiosity': {'precision': 1.0,
  'recall': 0.3888888888888889,
  'f1-score': 0.56,
  'support': 18},
 'gratitude': {'precision': 0.9864864864864865,
  'recall': 0.9605263157894737,
  'f1-score': 0.9733333333333333,
  'support': 76},
 'optimism': {'precision': 0.7272727272727273,
  'recall': 0.47058823529411764,
  'f1-score': 0.5714285714285714,
  'support': 17},
 'micro avg': {'precision': 0.9299363057324841,
  'recall': 0.7087378640776699,
  'f1-score': 0.8044077134986227,
  'support': 206},
 'macro avg': {'precision': 0.8864668717609895,
  'recall': 0.6125126506234327,
  'f1-score': 0.7112103174603175,
  'su

In [85]:
from rubrix.labeling.text_classification import Snorkel
from tqdm.auto import tqdm

def grid_search(weak_labels):
    snorkel_model = Snorkel(weak_labels)

    lrs = [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.]
    eps = [20, 40, 60, 80, 100]
    ths = [0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


    result = []
    for lr in tqdm(lrs):
        for ep in eps:
            snorkel_model = Snorkel(weak_labels, verbose=False)
            snorkel_model.fit(lr=lr, n_epochs=ep, progress_bar=False)
            for th in ths:
                metrics = snorkel_model.score(threshold=th)
                result.append((metrics["micro avg"]["f1-score"], metrics, lr, ep, th))
    return sorted(result, key=lambda x: x[0], reverse=True)    

In [86]:
snorkel_metrics = grid_search(weak_labels)

  0%|          | 0/13 [00:00<?, ?it/s]

/home/david/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/david/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/david/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [88]:
snorkel_metrics[0]

(0.8011049723756906,
 {'admiration': {'precision': 0.9696969696969697,
   'recall': 0.5925925925925926,
   'f1-score': 0.7356321839080459,
   'support': 54},
  'annoyance': {'precision': 0.9285714285714286,
   'recall': 0.7222222222222222,
   'f1-score': 0.8125000000000001,
   'support': 18},
  'approval': {'precision': 0.7058823529411765,
   'recall': 0.5217391304347826,
   'f1-score': 0.6,
   'support': 23},
  'curiosity': {'precision': 1.0,
   'recall': 0.3888888888888889,
   'f1-score': 0.56,
   'support': 18},
  'gratitude': {'precision': 0.9864864864864865,
   'recall': 0.9605263157894737,
   'f1-score': 0.9733333333333333,
   'support': 76},
  'optimism': {'precision': 0.7272727272727273,
   'recall': 0.47058823529411764,
   'f1-score': 0.5714285714285714,
   'support': 17},
  'micro avg': {'precision': 0.9294871794871795,
   'recall': 0.7038834951456311,
   'f1-score': 0.8011049723756906,
   'support': 206},
  'macro avg': {'precision': 0.8863183274947981,
   'recall': 0.609426

In [17]:
# Get records with the predictions from the label model to train a down-stream model
train_rb = rb.DatasetForTextClassification(label_model.predict())

# Copy label model predictions to annotation
for rec in train_rb:
    rec.annotation = [pred[0] for pred in rec.prediction if pred[1] > 0.5]

We extract the test set with manual annotations from our `WeakMultiLabels` object:

In [18]:
# Get records with manual annotations to use as test set for the down-stream model
test_rb = rb.DatasetForTextClassification(weak_labels.records(has_annotation=True))

We will use the convenient `DatasetForTextClassification.prepare_for_training()` method to create datasets optimized for training with the Hugging Face transformers library:

In [19]:
from datasets import DatasetDict

# Create dataset dictionary and shuffle training set
ds = DatasetDict(
    train=train_rb.prepare_for_training().shuffle(seed=42),
    test=test_rb.prepare_for_training()
)

Let us push the dataset to the Hub to share it with our colleagues.
It is also an easy way to outsource the training of the model to an environment with an accelerator, like Google Colab for example.

In [ ]:
# Push dataset for training our down-stream model to the HF hub
ds.push_to_hub("rubrix/go_emotions_training")

### Train a transformer downstream model

The following steps are basically a copy&paste from the amazing documentation of the [Hugging Face transformers](https://huggingface.co/docs/transformers) library.

First, we will load the tokenizer corresponding to our model, which we choose to be the [distilled version](https://huggingface.co/distilbert-base-uncased) of the infamous BERT.

<div class="alert alert-info">

Note

Since we will use a full-blown transformer as a downstream model (albeit a distilled one), we recommend executing the following code on a machine with a GPU, or in a Google Colab with a GPU backend enabled.
    
</div>

In [20]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Afterward, we tokenize our data:

In [ ]:
def tokenize_func(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the data
tokenized_ds = ds.map(tokenize_func, batched=True)

The transformer model expects our labels to follow a common multi-label format of binaries, so let us use [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) for this transformation.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Turn labels into multi-label format
mb = MultiLabelBinarizer()
mb.fit(ds["test"]["label"])

def binarize_labels(examples):
    return {"label": mb.transform(examples["label"])}

binarized_tokenized_ds = tokenized_ds.map(binarize_labels, batched=True)

Before we start the training, it is important to define our metric for the evaluation.
Here we settle on the commonly used micro averaged *F1* metric, but we will also keep track of the *F1 per label*, for a more in-depth error analysis afterward.

In [285]:
from datasets import load_metric
import numpy as np

# Define our metrics
metric = load_metric("f1", config_name="multilabel")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # apply sigmoid
    predictions = ( 1. / (1 + np.exp(-logits)) ) > 0.5
    
    # f1 micro averaged
    metrics = metric.compute(predictions=predictions, references=labels, average="micro")
    # f1 per label
    per_label_metric = metric.compute(predictions=predictions, references=labels, average=None)
    for label, f1 in zip(ds["train"].features["label"][0].names, per_label_metric["f1"]):
        metrics[f"f1_{label}"] = f1

    return metrics

Now we are ready to load our pretrained transformer model and prepare it for our task: multi-label text classification with 6 labels.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Init our down-stream model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    problem_type="multi_label_classification", 
    num_labels=6
)

The only thing missing for the training is the `Trainer` and its `TrainingArguments`.
To keep it simple, we mostly rely on the default arguments, that often work out of the box, but tweak a bit the batch size to train faster. 
We also checked that 2 epochs are enough for our rather small dataset.

In [283]:
from transformers import TrainingArguments

# Set our training arguments
training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    num_train_epochs=2,
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=16, 
)

In [286]:
from transformers import Trainer

# Init the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=binarized_tokenized_ds["train"],
    eval_dataset=binarized_tokenized_ds["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the down-stream model
trainer.train()

We achieved an micro averaged *F1* of abut 0.54, which is not perfect, but a good baseline for this challenging dataset.
When inspecting the *F1s per label*, we clearly see that the worst performing labels are the ones with the poorest heuristics in terms of accuracy and coverage, which comes to no surprise.

## Research topic dataset

After covering a multi-label emotion classification task, we will try to do the same for a multi-label classification task related to topic modeling.
In this dataset, research papers were classified with 6 non-exclusive labels based on their title and abstract.

We will try to classify the papers only based on the title, which is considerably harder, but allows us to quickly scan through the data and come up with heuristics.
See Appendix B for all the details of the minimal data preprocessing.

### Define rules

Let us start by downloading our preprocessed dataset from the Hugging Face Hub, and log it to Rubrix:

In [62]:
import rubrix as rb
from datasets import load_dataset

# Download preprocessed dataset
ds_rb = rb.read_datasets(
    load_dataset("rubrix/research_titles_multi-label", split="train"),
    task="TextClassification"
)

2022-04-30 13:31:59.022 | WARNING  | datasets.builder:_create_builder_config:378 - Using custom data configuration rubrix--research_titles_multi-label-91db2069895a8497
2022-04-30 13:31:59.028 | WARNING  | datasets.builder:download_and_prepare:531 - Reusing dataset parquet (/home/david/.cache/huggingface/datasets/parquet/rubrix--research_titles_multi-label-91db2069895a8497/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


In [63]:
# Log dataset to Rubrix to find good heuristics
rb.log(ds_rb, "research_titles")

  0%|          | 0/20972 [00:00<?, ?it/s]

20972 records logged to http://localhost:6900/datasets/rubrix/research_titles


BulkResponse(dataset='research_titles', processed=20972, failed=0)

After uploading the dataset, we can explore and inspect it to find good heuristic rules.
For this we highly recommend the dedicated [*Define rules* mode](../reference/webapp/define_rules.md) of the Rubrix web app, that allows you to quickly iterate over heuristic rules, compute their metrics and save them.

Here we copy our rules found via the web app to the notebook for you to easily follow along the tutorial.

In [64]:
from rubrix.labeling.text_classification import Rule

# Define our heuristic rules (can probably be improved)
rules = [
    Rule("stock*", "Quantitative Finance"),
    Rule("*asset*", "Quantitative Finance"),
    Rule("trading", "Quantitative Finance"),
    Rule("finance", "Quantitative Finance"),
    Rule("pric*", "Quantitative Finance"),
    Rule("economy", "Quantitative Finance"),
    Rule("deep AND neural AND network*", "Computer Science"),
    Rule("convolutional", "Computer Science"),
    Rule("memor* AND (design* OR network*)", "Computer Science"),
    Rule("system* AND design*", "Computer Science"),
    Rule("allocat* AND *net*", "Computer Science"),
    Rule("program", "Computer Science"),
    Rule("classification* AND (label* OR deep)", "Computer Science"),
    Rule("scattering", "Physics"),
    Rule("astro*", "Physics"),
    Rule("material*", "Physics"),
    Rule("spin", "Physics"),
    Rule("magnetic", "Physics"),
    Rule("optical", "Physics"),
    Rule("ray", "Physics"),
    Rule("entangle*", "Physics"),
    Rule("*algebra*", "Mathematics"),
    Rule("manifold* AND (NOT learn*)", "Mathematics"),
    Rule("equation", "Mathematics"),
    Rule("spaces", "Mathematics"), 
    Rule("operators", "Mathematics"), 
    Rule("regression", "Statistics"),
    Rule("bayes*", "Statistics"),
    Rule("estimation", "Statistics"),
    Rule("mixture", "Statistics"),
    Rule("gaussian", "Statistics"),
    Rule("gene", "Quantitative Biology"),
]

We go on and apply these heuristic rules to our dataset creating our weak label matrix.
As mentioned in the [GoEmotions](#goemotions) section, the weak label matrix will have 3 dimensions and values of -1, 0 and 1.

In [65]:
from rubrix.labeling.text_classification import WeakMultiLabels

# Compute the weak labels for our dataset given the rules
# If your dataset already contains rules you can omit the rules argument.
weak_labels = WeakMultiLabels("research_titles", rules=rules)

Preparing rules:   0%|          | 0/32 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/20972 [00:00<?, ?it/s]

Filling weak label matrix:   0%|          | 0/20972 [00:00<?, ?it/s]

Let us get an overview of the our heuristics and how they perform:

In [66]:
# Check coverage/precision of our rules
weak_labels.summary()

,label,coverage,annotated_coverage,overlaps,correct,incorrect,precision
stock*,{Quantitative Finance},0.000954,0.000715,0.000334,3,0,1.000000
*asset*,{Quantitative Finance},0.000477,0.000715,0.000286,3,0,1.000000
trading,{Quantitative Finance},0.000954,0.000238,0.000191,1,0,1.000000
finance,{Quantitative Finance},0.000048,0.000238,0.000000,1,0,1.000000
pric*,{Quantitative Finance},0.003433,0.003337,0.000715,9,5,0.642857
economy,{Quantitative Finance},0.000238,0.000238,0.000000,1,0,1.000000
deep AND neural AND network*,{Computer Science},0.009155,0.010250,0.002909,32,11,0.744186
convolutional,{Computer Science},0.010109,0.009297,0.002241,32,7,0.820513
memor* AND (design* OR network*),{Computer Science},0.001383,0.002145,0.000286,9,0,1.000000
system* AND design*,{Computer Science},0.001144,0.002384,0.000238,9,1,0.900000


### Create training set

When we are happy with our heuristics, it is time to combine them and compute weak labels for the training of our downstream model.
As for the "GoEmotions" dataset, we will use the simple `MajorityVoter`.

In [67]:
from rubrix.labeling.text_classification import MajorityVoter

# Use the majority voter as the label model
label_model = MajorityVoter(weak_labels)

From our label model we get the training records together with its weak labels and probabilities.
Since we are going to train an sklearn model, we will put the records in a pandas DataFrame that generally has a good integration with the sklearn ecosystem.

In [68]:
label_model.score()

{'Computer Science': {'precision': 0.811965811965812,
  'recall': 0.43577981651376146,
  'f1-score': 0.5671641791044777,
  'support': 218},
 'Mathematics': {'precision': 0.8592233009708737,
  'recall': 0.6654135338345865,
  'f1-score': 0.75,
  'support': 266},
 'Physics': {'precision': 0.9078341013824884,
  'recall': 0.8140495867768595,
  'f1-score': 0.8583877995642701,
  'support': 242},
 'Quantitative Biology': {'precision': 0.8571428571428571,
  'recall': 0.5454545454545454,
  'f1-score': 0.6666666666666665,
  'support': 11},
 'Quantitative Finance': {'precision': 0.7727272727272727,
  'recall': 0.9444444444444444,
  'f1-score': 0.85,
  'support': 18},
 'Statistics': {'precision': 0.7705627705627706,
  'recall': 0.6980392156862745,
  'f1-score': 0.7325102880658435,
  'support': 255},
 'micro avg': {'precision': 0.8375,
  'recall': 0.6633663366336634,
  'f1-score': 0.7403314917127072,
  'support': 1010},
 'macro avg': {'precision': 0.8299093524586793,
  'recall': 0.6838635237850786,


#### Compare to Snorkel model

In [70]:
metrics = grid_search(weak_labels)

  0%|          | 0/13 [00:00<?, ?it/s]

/home/david/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/david/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/david/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [72]:
metrics

[(0.7391064533921677,
  {'Computer Science': {'precision': 0.8050847457627118,
    'recall': 0.43577981651376146,
    'f1-score': 0.5654761904761905,
    'support': 218},
   'Mathematics': {'precision': 0.8592233009708737,
    'recall': 0.6654135338345865,
    'f1-score': 0.75,
    'support': 266},
   'Physics': {'precision': 0.9036697247706422,
    'recall': 0.8140495867768595,
    'f1-score': 0.8565217391304347,
    'support': 242},
   'Quantitative Biology': {'precision': 0.8571428571428571,
    'recall': 0.5454545454545454,
    'f1-score': 0.6666666666666665,
    'support': 11},
   'Quantitative Finance': {'precision': 0.7727272727272727,
    'recall': 0.9444444444444444,
    'f1-score': 0.85,
    'support': 18},
   'Statistics': {'precision': 0.7672413793103449,
    'recall': 0.6980392156862745,
    'f1-score': 0.7310061601642709,
    'support': 255},
   'micro avg': {'precision': 0.8343711083437111,
    'recall': 0.6633663366336634,
    'f1-score': 0.7391064533921677,
    'suppor

In [58]:
train_df = rb.DatasetForTextClassification(label_model.predict()).to_pandas()

Before training our model, we need to extract the training labels from the label model predictions and transform them into a multi-label compatible format.

In [59]:
# Create labels in multi-label format
train_df["label"] = train_df.prediction.map(
    lambda x: [
        # we will use a threshold of 0.5 for the probability
        {p[0]: int(p[1] > 0.5) for p in x}[label] 
        for label in weak_labels.labels
    ]
)

Now, let us define our downstream model and train it.

We will use the [scikit-multilearn library](http://scikit.ml/) to wrap a multinomial **Naive Bayes classifier** that is suitable for classification with discrete features (e.g., word counts for text classification).
The `BinaryRelevance` class transforms the multi-label problem with L labels into L single-label binary classification problems, so in the end we will automatically fit L naive bayes classifiers to our data.

The features for our classifier will be the counts of different word [n-grams](https://en.wikipedia.org/wiki/N-gram): that is, for each example we count the number of contiguous sequences of *n* words, where n goes from 1 to 5.
We extract these features with the `CountVectorizer`.

Finally, we will put our feature extractor and multi-label classifier in a sklearn pipeline that makes fitting and scoring the model a breeze.

In [147]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Define our down-stream model
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', BinaryRelevance(MultinomialNB()))
])

Training the model is as easy as calling the `fit` method on the our pipeline, and provide our training text and training labels.

In [ ]:
import numpy as np

# Fit the down-stream classifier
classifier.fit(
    X=train_df.text,
    y=np.array(train_df.label.tolist()),
)

To score our trained model, we retrieve its predictions of the test set and use sklearn's `classification_report` to get all important classification metrics in a nicely formatted string.

In [86]:
# Get predictions for test set
predictions = classifier.predict(
    X=[rec.text for rec in weak_labels.records(has_annotation=True)]
)

In [77]:
from sklearn.metrics import classification_report

# Compute metrics
print(classification_report(
    weak_labels.annotation(), 
    predictions, 
    target_names=weak_labels.labels
))

                      precision    recall  f1-score   support

    Computer Science       0.82      0.26      0.40      1740
         Mathematics       0.71      0.64      0.67      1141
             Physics       0.81      0.70      0.75      1186
Quantitative Biology       1.00      0.01      0.02       109
Quantitative Finance       0.44      0.09      0.15        45
          Statistics       0.48      0.71      0.57      1069

           micro avg       0.66      0.53      0.59      5290
           macro avg       0.71      0.40      0.43      5290
        weighted avg       0.73      0.53      0.56      5290
         samples avg       0.66      0.56      0.59      5290



We obtain a micro averaged F1 score of around 0.59, which again is not perfect but can serve as a decent baseline for future improvements.
Looking at the F1 per label, we see that the main problem is the recall of our heuristics and we should either define more of them, or try to find more general ones. 

## Summary

In this tutorial we saw how you can use *Rubrix* to tackle multi-label text classification problems with weak supervision.
We showed you how to train two downstream models on two different multi-label datasets using the discovered heuristics.

For the emotion classification task, we trained a full-blown transformer model with Hugging Face, while for the topic classification task, we relied on a more lightweight Bayes classifier from sklearn.
Although the results are not perfect, they can serve as a good baseline for future improvements.

So the next time you encounter a multi-label classification problem, maybe try out weak supervision with *Rubrix* and save some time for your annotation team 😀.

## Next steps

**⭐ Star Rubrix [Github repo](https://github.com/recognai/rubrix) to stay updated.**

**📚 [Rubrix documentation](https://docs.rubrix.ml) for more guides and tutorials.**

**🙋‍♀️ Join the Rubrix community! A good place to start is the [discussion forum](https://github.com/recognai/rubrix/discussions).**

## Appendix A

This appendix summarizes the preprocessing steps for our curated *GoEmotions* dataset.
The goal was to limit the labels, and down-sample single-label annotations to move the focus to multi-label outputs.

In [1]:
# load original dataset and check label frequencies

import pandas as pd
import datasets

go_emotions = datasets.load_dataset("go_emotions")
df = go_emotions["test"].to_pandas()

def int2str(i):
    #return int(i)
    return go_emotions["train"].features["labels"].feature.int2str(int(i))

label_freq = []
idx_multi = df.labels.map(lambda x: len(x) > 1)
df["is_single"] = df.labels.map(lambda x: 0 if len(x) > 1 else 1) 
df[idx_multi].labels.map(lambda x: [label_freq.append(int(l)) for l in x])
pd.Series(label_freq).value_counts();

In [22]:
# limit labels, down-sample single-label annotations and create Rubrix records

import rubrix as rb

def create(split: str) -> pd.DataFrame:
    df = go_emotions[split].to_pandas()
    df["is_single"] = df.labels.map(lambda x: 0 if len(x) > 1 else 1)
    
    #['admiration', 'approval', 'annoyance', 'gratitude', 'curiosity', 'optimism', 'amusement']
    idx_most_common = df.labels.map(lambda x: all([int(label) in [0, 4, 3, 15, 7, 15, 20] for label in x]))
    df_multi = df[(df.is_single == 0) & idx_most_common]
    df_single = df[idx_most_common].sample(3*len(df_multi), weights="is_single", axis=0, random_state=42)
    return pd.concat([df_multi, df_single]).sample(frac=1, random_state=42)

def make_records(row, is_train: bool) -> rb.TextClassificationRecord:
    annotation = [int2str(i) for i in row.labels] if not is_train else None
    return rb.TextClassificationRecord(
        inputs=row.text,
        annotation=annotation,
        multi_label=True,
        id=row.id,
    )

train_recs = create("train").apply(make_records, axis=1, is_train=True)
test_recs = create("test").apply(make_records, axis=1, is_train=False)

records = train_recs.to_list() + test_recs.tolist()

In [27]:
# publish dataset in the Hub

ds_rb = rb.DatasetForTextClassification(records).to_datasets()

ds_rb.push_to_hub("rubrix/go_emotions_multi-label", private=True)

## Appendix B

This appendix summarizes the minimal preprocessing done to [this multi-label classification dataset](https://www.kaggle.com/shivanandmn/multilabel-classification-dataset) from Kaggle.
You can download the original data (`train.csv`) following the Kaggle link.

The preprocessing consists of extracting only the title from the research paper, and split the data into a train and validation set.

In [16]:
# Extact the title and split the data

import pandas as pd
import rubrix as rb
from sklearn.model_selection import train_test_split

df = pd.read_csv("train.csv")

_, test_id = train_test_split(df.ID, test_size=0.2, random_state=42)

labels = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]
def make_record(row):
    annotation = [label for label in labels if row[label] == 1]
    return rb.TextClassificationRecord(
        inputs=row.TITLE,
        # inputs={"title": row.TITLE, "abstract": row.ABSTRACT},
        annotation=annotation if row.ID in test_id else None,
        multi_label=True,
        id=row.ID,
    )

records = df.apply(make_record, axis=1)

In [ ]:
# publish the dataset in the Hub

dataset_rb = rb.DatasetForTextClassification(records.tolist())

dataset_rb.to_datasets().push_to_hub("rubrix/research_titles_multi-label")